In [ ]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pykrx import stock
sns.set(font='NaNumBarunGothic')

import tensorflow as tf


from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.max_columns = None

In [ ]:

df_for_stock_num = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/REALREALFINALJAM.csv')

관리종목지정요건 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/관리종목지정요건.csv')
관리종목지정요건.head(10)

def make_preprocessing(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4]})")


  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df =df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace = True)
  df = df.reset_index(drop= True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)




  # 문자열 float로 변경
  for i in df.columns[7:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '',x))
    df[i] = df[i].astype('float')



  # # null인 경우 0으로 채운다. -> o_score 도출을 위해서(널값이 많이 나오지는 않았다.)
  # for i in df.columns:
  #   if '당기순이익' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '유형,임대주택자산감가상각비' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '무형자산상각비' in i:
  #     df[i].fillna(0, inplace=True)
  #   if '법인세비용 ' in i:
  #     df[i].fillna(0, inplace=True)
  # 종목카드 6자리로 채워주기
  df['Stock'] = df['Stock'].astype('int')
  df['Stock'] = df['Stock'].astype('str')
  df['Stock'] = df['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

  # 널값 1개 있는것 최빈값으로 채워주기
  df['감사의견 (2020)'].fillna('적정', inplace=True)

  return df

df = make_preprocessing(관리종목지정요건)

In [ ]:
print(df['감사의견 (2018)'].value_counts())
print(df['감사의견 (2019)'].value_counts())
print(df['감사의견 (2020)'].value_counts())
print(df['감사의견 (2021)'].value_counts())

적정              1371
감사미필              14
한정(감사범위제한)         7
의견거절(감사범위제한)       1
Name: 감사의견 (2018), dtype: int64
적정              1411
한정(감사범위제한)         5
감사미필               5
의견거절(감사범위제한)       1
Name: 감사의견 (2019), dtype: int64
적정              1443
한정(감사범위제한)         2
의견거절(감사범위제한)       1
Name: 감사의견 (2020), dtype: int64
적정    1434
Name: 감사의견 (2021), dtype: int64


In [ ]:

# 종목카드 6자리로 채워주기
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].astype('int')
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].astype('str')
df_for_stock_num['Stock'] = df_for_stock_num['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

tmp = []
관리종목_df = pd.DataFrame()
for i in df['Stock'].unique():
  if i in df_for_stock_num['Stock'].unique():
    idx = list(df[df['Stock']==i].index.values).pop()
    tmp.append(idx)
관리종목_df  =pd.DataFrame(df, index = tmp).reset_index(drop=True)



관리종목_df

,KIS,Stock,Name,감사의견 (2018),감사의견 (2019),감사의견 (2020),감사의견 (2021),매출액 (2018/Annual),법인세비용차감전계속사업이익(손실) (2018/Annual),영업이익(손실) (2018/Annual),매출액 (2019/Annual),법인세비용차감전계속사업이익(손실) (2019/Annual),영업이익(손실) (2019/Annual),매출액 (2020/Annual),법인세비용차감전계속사업이익(손실) (2020/Annual),영업이익(손실) (2020/Annual),매출액 (2021/Annual),법인세비용차감전계속사업이익(손실) (2021/Annual),영업이익(손실) (2021/Annual)
0,036644,389500,(주)에스비비테크,적정,한정(감사범위제한),적정,적정,9.218357e+09,-1.917403e+09,-1.088044e+09,6.019507e+09,-4.788196e+09,-3.769930e+09,6.606806e+09,-3.309699e+09,-2.616470e+09,6.779379e+09,-2.817270e+09,-2.227144e+09
1,497730,296640,(주)이노룰스,적정,적정,적정,적정,9.313410e+09,1.478880e+08,4.809880e+08,1.340630e+10,1.191682e+09,2.768827e+09,1.063404e+10,1.024106e+09,1.187901e+09,1.597968e+10,3.324217e+09,3.314891e+09
2,499459,417970,모델솔루션(주),적정,적정,적정,적정,5.559806e+10,9.218748e+09,9.433205e+09,6.056418e+10,9.309068e+09,9.332657e+09,5.501846e+10,2.941833e+09,6.870288e+09,6.007888e+10,9.146970e+09,8.526295e+09
3,HS8995,314140,(주)알피바이오,적정,적정,적정,적정,6.476281e+10,5.851262e+09,6.382369e+09,6.985516e+10,3.351179e+09,3.959920e+09,9.792265e+10,4.209396e+09,4.157063e+09,1.149640e+11,4.290818e+09,5.879847e+09
4,867055,129920,(주)대성하이텍,적정,적정,적정,적정,8.863754e+10,2.019903e+09,4.717809e+09,7.314135e+10,3.214249e+09,5.169880e+09,7.265322e+10,-1.170560e+10,-8.452512e+09,1.027162e+11,9.244219e+09,1.169168e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),적정,적정,적정,적정,2.374177e+10,3.762250e+09,4.254804e+09,2.144421e+10,1.320041e+09,1.734016e+09,3.055471e+10,8.408703e+09,8.915481e+09,2.951487e+10,1.011052e+10,1.688230e+09
1275,125028,046970,(주)우리로,적정,적정,적정,적정,3.197440e+10,-3.554180e+08,2.718326e+09,1.015362e+11,-8.829858e+09,2.323770e+08,9.735110e+10,-1.162556e+10,-2.502020e+08,9.659889e+10,1.335246e+10,3.130126e+09
1276,383046,113810,디젠스(주),적정,적정,적정,적정,3.723606e+10,-8.257150e+09,-2.946957e+09,3.968948e+10,2.659499e+09,2.632887e+09,4.959402e+10,-7.319745e+09,-3.599090e+08,4.927206e+10,3.523810e+08,-6.435750e+08
1277,F11411,155650,와이엠씨(주),적정,적정,적정,적정,2.315697e+11,1.871956e+10,1.980928e+10,1.623188e+11,-3.341818e+09,-5.169211e+09,1.456117e+11,1.086622e+10,1.025215e+10,1.784667e+11,1.237523e+10,1.139106e+10


In [ ]:
# 감사의견 (2020) 한개만 널값이 나온다.
관리종목_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   KIS                               1279 non-null   object 
 1   Stock                             1279 non-null   object 
 2   Name                              1279 non-null   object 
 3   감사의견 (2018)                       1279 non-null   object 
 4   감사의견 (2019)                       1279 non-null   object 
 5   감사의견 (2020)                       1279 non-null   object 
 6   감사의견 (2021)                       1279 non-null   object 
 7   매출액 (2018/Annual)                 1279 non-null   float64
 8   법인세비용차감전계속사업이익(손실) (2018/Annual)  1279 non-null   float64
 9   영업이익(손실) (2018/Annual)            1279 non-null   float64
 10  매출액 (2019/Annual)                 1279 non-null   float64
 11  법인세비용차감전계속사업이익(손실) (2019/Annual)  1279 non-null   float64
 12  영업이익(손

In [ ]:
df_base = 관리종목_df[['Stock']]
df_2018 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2018' in col]]], axis=1)
df_2019 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2019' in col]]], axis=1)
df_2020 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2020' in col]]], axis=1)
df_2021 = pd.concat([df_base,관리종목_df[[col for col in 관리종목_df.columns if '2021' in col]]], axis=1)

In [ ]:
df_2018

,Stock,감사의견 (2018),매출액 (2018/Annual),법인세비용차감전계속사업이익(손실) (2018/Annual),영업이익(손실) (2018/Annual)
0,389500,적정,9.218357e+09,-1.917403e+09,-1.088044e+09
1,296640,적정,9.313410e+09,1.478880e+08,4.809880e+08
2,417970,적정,5.559806e+10,9.218748e+09,9.433205e+09
3,314140,적정,6.476281e+10,5.851262e+09,6.382369e+09
4,129920,적정,8.863754e+10,2.019903e+09,4.717809e+09
...,...,...,...,...,...
1274,950110,적정,2.374177e+10,3.762250e+09,4.254804e+09
1275,046970,적정,3.197440e+10,-3.554180e+08,2.718326e+09
1276,113810,적정,3.723606e+10,-8.257150e+09,-2.946957e+09
1277,155650,적정,2.315697e+11,1.871956e+10,1.980928e+10


In [ ]:
df_2019

,Stock,감사의견 (2019),매출액 (2019/Annual),법인세비용차감전계속사업이익(손실) (2019/Annual),영업이익(손실) (2019/Annual)
0,389500,한정(감사범위제한),6.019507e+09,-4.788196e+09,-3.769930e+09
1,296640,적정,1.340630e+10,1.191682e+09,2.768827e+09
2,417970,적정,6.056418e+10,9.309068e+09,9.332657e+09
3,314140,적정,6.985516e+10,3.351179e+09,3.959920e+09
4,129920,적정,7.314135e+10,3.214249e+09,5.169880e+09
...,...,...,...,...,...
1274,950110,적정,2.144421e+10,1.320041e+09,1.734016e+09
1275,046970,적정,1.015362e+11,-8.829858e+09,2.323770e+08
1276,113810,적정,3.968948e+10,2.659499e+09,2.632887e+09
1277,155650,적정,1.623188e+11,-3.341818e+09,-5.169211e+09


In [ ]:
df_2020

,Stock,감사의견 (2020),매출액 (2020/Annual),법인세비용차감전계속사업이익(손실) (2020/Annual),영업이익(손실) (2020/Annual)
0,389500,적정,6.606806e+09,-3.309699e+09,-2.616470e+09
1,296640,적정,1.063404e+10,1.024106e+09,1.187901e+09
2,417970,적정,5.501846e+10,2.941833e+09,6.870288e+09
3,314140,적정,9.792265e+10,4.209396e+09,4.157063e+09
4,129920,적정,7.265322e+10,-1.170560e+10,-8.452512e+09
...,...,...,...,...,...
1274,950110,적정,3.055471e+10,8.408703e+09,8.915481e+09
1275,046970,적정,9.735110e+10,-1.162556e+10,-2.502020e+08
1276,113810,적정,4.959402e+10,-7.319745e+09,-3.599090e+08
1277,155650,적정,1.456117e+11,1.086622e+10,1.025215e+10


In [ ]:
df_2021

,Stock,감사의견 (2021),매출액 (2021/Annual),법인세비용차감전계속사업이익(손실) (2021/Annual),영업이익(손실) (2021/Annual)
0,389500,적정,6.779379e+09,-2.817270e+09,-2.227144e+09
1,296640,적정,1.597968e+10,3.324217e+09,3.314891e+09
2,417970,적정,6.007888e+10,9.146970e+09,8.526295e+09
3,314140,적정,1.149640e+11,4.290818e+09,5.879847e+09
4,129920,적정,1.027162e+11,9.244219e+09,1.169168e+10
...,...,...,...,...,...
1274,950110,적정,2.951487e+10,1.011052e+10,1.688230e+09
1275,046970,적정,9.659889e+10,1.335246e+10,3.130126e+09
1276,113810,적정,4.927206e+10,3.523810e+08,-6.435750e+08
1277,155650,적정,1.784667e+11,1.237523e+10,1.139106e+10


In [ ]:
def make_관리종목요건(df):

  for col in df.columns:
    if '감사의견' in col:
      감사의견 = col
    if '매출액' in col:
      매출액 = col
    if '법인세비용차감전계속사업이익(손실)' in col:
      법인세비용차감전계속사업이익_손실 = col
    if '영업이익' in col:
      영업이익 = col
    

  관리종목_df = pd.DataFrame()
  관리종목_df['Stock'] = df['Stock']
  # 감사의견 
  # 적정 : 3
  # 감사미필 : 2
  # 한정(감사범위제한) : 1
  # 의견거절(감사범위제한) : 0
  def 감사_labeling(x):
    if x=='적정':
      return 1
    elif x=='감사미필':
      return 2
    elif x=='한정(감사범위제한)': # 관리종목에 지정되는 기업
      return 1
    else: # 거절 또는 부적정 판정을 받은 기업의 경우 곧바로 상장 폐지
      return 0

  # 관리종목 라벨링
  관리종목_df['관리종목요건_감사의견'] = df[f'{감사의견}'].apply(감사_labeling)
  관리종목_df['관리종목요건_매출액'] = df.apply(lambda x: 1 if x[f'{매출액}']>=3000000000 else 0, axis=1)
  관리종목_df['관리종목요건_영업손실'] = df.apply(lambda x: 1 if x[f'{영업이익}']>0 else 0, axis=1)
  관리종목_df['관리종목요건_법인세비용차감전계속사업손실'] = df.apply(lambda x:1 if x[f'{법인세비용차감전계속사업이익_손실}']>0 else 0, axis=1)

  return 관리종목_df




In [ ]:
관리종목지정요건_2018 = make_관리종목요건(df_2018)
관리종목지정요건_2019 = make_관리종목요건(df_2019)
관리종목지정요건_2020 = make_관리종목요건(df_2020)
관리종목지정요건_2021 = make_관리종목요건(df_2021)

In [ ]:
관리종목지정요건_2018

,Stock,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실
0,389500,3,1,0,0
1,296640,3,1,1,1
2,417970,3,1,1,1
3,314140,3,1,1,1
4,129920,3,1,1,1
...,...,...,...,...,...
1274,950110,3,1,1,1
1275,046970,3,1,1,0
1276,113810,3,1,0,0
1277,155650,3,1,1,1


In [ ]:
new_df = pd.concat([관리종목지정요건_2018, 관리종목지정요건_2019, 관리종목지정요건_2020, 관리종목지정요건_2021], axis=0).reset_index(drop = True)
new_df

,Stock,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실
0,389500,3,1,0,0
1,296640,3,1,1,1
2,417970,3,1,1,1
3,314140,3,1,1,1
4,129920,3,1,1,1
...,...,...,...,...,...
5111,950110,3,1,1,1
5112,046970,3,1,1,1
5113,113810,3,1,0,1
5114,155650,3,1,1,1


In [ ]:
new_df.drop('Stock', axis=1,inplace=True)

# baseline2.csv 만들기
- baseline1.csv + 관리종목지정요건 컬럼 4개
- 지정요건 추가해주기

In [ ]:
baseline1 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/baseline1.csv')
baseline2 = pd.concat([baseline1, new_df], axis=1)
baseline2

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1
5112,46970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1


In [ ]:
baseline2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5116 entries, 0 to 5115
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Stock                  5116 non-null   int64  
 1   총자산증가율                 5116 non-null   float64
 2   자기자본증가율                5116 non-null   float64
 3   매출액증가율                 5116 non-null   float64
 4   총자본영업이익율               5116 non-null   float64
 5   총자본순이익율                5116 non-null   float64
 6   자본금순이익율                5116 non-null   float64
 7   매출액순이익율                5116 non-null   float64
 8   매출액영업이익율               5116 non-null   float64
 9   자기자본비율                 5116 non-null   float64
 10  부채비율                   5116 non-null   float64
 11  유보액/납입자본비율             5116 non-null   float64
 12  총자본회전율                 5116 non-null   float64
 13  자기자본회전율                5116 non-null   float64
 14   당기순이익(손실)             5116 non-null   float64
 15  영업활동

In [ ]:
baseline2.to_csv('baseline2.csv', index=False)

# 관리종목지정요건 컬럼 시각화

In [ ]:
baseline2

,Stock,총자산증가율,자기자본증가율,매출액증가율,총자본영업이익율,총자본순이익율,자본금순이익율,매출액순이익율,매출액영업이익율,자기자본비율,부채비율,유보액/납입자본비율,총자본회전율,자기자본회전율,당기순이익(손실),영업활동으로인한현금흐름,성장성,안정성,활동성,수익성,현금흐름,Z_score,부실기업판별_Z_score,부도확률_O_score,F_score,K_score,부실기업판별_K_score,자본잠식,벌금,불성실공시,소송,영업정지,특허,투자주의환기종목,업종,관리종목여부,관리종목요건_감사의견,관리종목요건_매출액,관리종목요건_영업손실,관리종목요건_법인세비용차감전계속사업손실
0,389500,46.03,-39.46,2.26,-5.23,-9.21,-168.76,-20.80,-11.80,20.22,394.44,340.07,0.44,1.39,-1.917031e+09,6.075800e+07,2.0,1.0,2.0,0.0,1.0,0.000000,3.0,0.023564,1.0,14.593801,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,0,0
1,296640,81.84,215.02,-6.23,3.88,1.43,68.58,1.90,5.16,40.95,144.17,2371.70,0.75,2.16,1.770280e+08,1.148444e+09,1.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.006766,1.0,17.328700,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1
2,417970,23.53,44.46,9.65,25.30,19.34,768.48,12.97,16.97,47.94,108.60,2005.45,1.49,3.33,7.210181e+09,5.469434e+09,2.0,2.0,2.0,5.0,2.0,0.000000,3.0,0.000343,1.0,26.200702,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
3,314140,38.75,31.69,-5.02,8.28,5.66,146.00,6.74,9.85,58.82,70.00,1558.22,0.84,1.40,4.363446e+09,6.995499e+09,1.0,3.0,2.0,3.0,2.0,0.000000,3.0,0.000461,1.0,21.578064,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
4,129920,8.36,26.30,7.67,3.96,1.57,162.88,2.10,5.32,38.03,162.97,4012.35,0.74,2.10,1.865670e+09,8.755060e+08,2.0,2.0,2.0,3.0,2.0,0.000000,3.0,0.001602,1.0,20.705556,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5111,950110,78.37,-3.08,-3.40,0.92,3.84,34.32,23.90,5.72,17.63,467.08,117.88,0.16,0.70,7.052749e+09,-8.972330e+10,1.0,0.0,1.0,4.0,1.0,0.965841,2.0,0.005466,1.0,15.903376,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,1,1,1
5112,46970,2.98,99.94,-0.77,5.31,22.63,88.70,13.82,3.24,64.99,53.87,154.51,1.64,3.31,1.335246e+10,-2.713149e+09,1.0,2.0,2.0,4.0,1.0,3.323928,0.0,0.000051,1.0,15.822916,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,1.0,3,1,1,1
5113,113810,-1.44,-4.48,-0.65,-1.02,0.04,0.16,0.05,-1.31,32.45,208.15,24.23,0.78,2.38,2.625700e+07,2.652719e+09,0.0,1.0,2.0,2.0,2.0,0.806495,2.0,0.002698,1.0,14.801748,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3,1,0,1
5114,155650,4.66,7.88,22.56,10.32,8.13,89.91,5.03,6.38,80.72,23.88,897.93,1.62,2.03,8.979293e+09,6.462723e+09,2.0,3.0,2.0,3.0,2.0,7.915767,0.0,0.000030,1.0,31.194817,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3,1,1,1


40

In [ ]:
# 발행주식수

#